<a href="https://colab.research.google.com/github/AgustinCocciardi/IA_Tps/blob/main/Copy_Trabajo_Pr%C3%A1ctico_ML_Regresi%C3%B3n_Clasificaci%C3%B3n_Multiclase_Frutas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico ML - Regresión Logística - Clasificación Multiclase - Frutas
---



Este dataset lo obtuvimos del siguiente sitio: https://www.muratkoklu.com/datasets/

Clasificación de frutos de dátiles en variedades genéticas.
En todo el mundo se cultiva una gran cantidad de frutas, cada una de las cuales tiene varios tipos. Los factores que determinan el tipo de fruto son las características de la apariencia externa como el color, la longitud, el diámetro y la forma. La apariencia externa de los frutos es un determinante importante del tipo de fruto. Determinar la variedad de frutas observando su apariencia externa puede requerir experiencia, lo que lleva mucho tiempo y requiere un gran esfuerzo. El objetivo de este estudio es clasificar los tipos de dátiles, es decir, Barhee, Deglet Nour, Sukkary, Rotab Mozafati, Ruthana, Safawi y Sagai, utilizando tres métodos diferentes de aprendizaje automático. Se extrajeron un total de 34 características, incluidas características morfológicas, forma y color.

# Contenido Teórico y Práctico para Desarrollo del Trabajo Práctico

**Importación Librerías**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# LINK SKLEARN LOGISTIC REGRESSION: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

**Carga Dataset**

In [ ]:
dataset = pd.read_csv('/content/04_Fruit_Datasets.csv')

**Análisis Dataset**

In [ ]:
print('FRUITS DATASET - HEAD: \n', dataset.head(5))

In [ ]:
print('FRUITS DATASET - SHAPE: ', dataset.shape)

FRUITS DATASET - SHAPE:  (898, 35)


In [ ]:
print('FRUITS DATASET - INFO: \n', dataset.info())

In [ ]:
print('FRUITS DATASET - STATISTICS: \n', dataset.describe())

In [ ]:
print('FRUITS DATASET - CANTIDAD DE INSTANCIAS POR CLASE: \n', dataset.Class.value_counts())

**Preprocesamiento de Datos**

In [ ]:
# Numerizamos valores en cadena de texto a valor numérico.
# Acomodamos valores de features para facilitar su operatoria.
classes = {'BERHI': 0, 'IRAQI': 1, 'SOGAY': 2, 'DEGLET': 3, 'ROTANA': 4, 'SAFAVI': 5, 'DOKOL': 6}

# Reemplazamos descripciones de feature por su correspondiente valor numérico.
dataset.Class = [classes[item] for item in dataset.Class]
print('FRUITS DATASET - CLASS FEATURE NUMERIZED: \n', dataset)

In [ ]:
print('FRUITS DATASET - CANTIDAD DE INSTANCIAS POR CLASE: \n', dataset.Class.value_counts())

In [ ]:
classes = dataset.Class.values
unique, counts = np.unique(classes, return_counts=True)

plt.bar(unique,counts)
plt.title('Class Frequency')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.show()

**Análisis Datos**

Coeficientes Correlación (Ver referencia: [correlation matrix](https://www.geeksforgeeks.org/create-a-correlation-matrix-using-python/))
Un coeficiente de correlación (normalmente denominado r) es un número único que describe el alcance de la relación lineal entre dos variables. Un valor de +1 indica una linealidad perfecta (las dos variables se mueven juntas, como "altura en pulgadas" y "altura en centímetros"). Un valor de r = 0 indica que no hay correlación (las variables son independientes) y r = -1 indica que las variables están inversamente correlacionadas (un aumento en una variable se asocia con una disminución en la otra).

In [ ]:
cormat = dataset.corr()
print('FRUITS DATASET - CORRELATION MATRIX: \n', round(cormat,2))
sns.heatmap(cormat);

In [ ]:
# Como primer análisis, seleccionamos aquellas features que tienen un valor de correlación cercano a 1.
sns.pairplot(dataset[[
 'AREA',
 'PERIMETER',
 'MAJOR_AXIS',
 'MINOR_AXIS',
 'EQDIASQ',
 'CONVEX_AREA',
 'EXTENT'
 ]]);

Otra técnica utilizada para identificar relación entre features para seleccionar las más relevantes podemos investigar [mutual_info de scikit learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression) y [ejemplo](https://guhanesvar.medium.com/feature-selection-based-on-mutual-information-gain-for-classification-and-regression-d0f86ea5262a)

In [ ]:
x_features = dataset.drop(labels=['Class'], axis=1)
x_features

In [ ]:
y_features = dataset['Class']
y_features

In [ ]:
x_train_mutual_info, x_test_mutual_info, y_train_mutual_info, y_test_mutual_info = train_test_split(x_features, y_features, test_size=0.2, random_state=0, shuffle=True)

In [ ]:
from sklearn.feature_selection import mutual_info_regression
mutual_info = mutual_info_regression(x_train_mutual_info, y_train_mutual_info)
print('FRUITS DATASET - MUTUAL INFO: \n', mutual_info)

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = x_train_mutual_info.columns
mutual_info.sort_values(ascending=False)

In [ ]:
mutual_info.sort_values(ascending=False).plot.bar(figsize=(15,5))

**Determinación Features Cols**

In [ ]:
# Determinamos las features que vamos a utilizar para analizar y operar.
feature_cols = [
 'MINOR_AXIS',
'MAJOR_AXIS',
'AREA',
'PERIMETER',
'SHAPEFACTOR_2',
'MeanRR',
'MeanRG',
'MeanRB',
'SkewRR',
'SkewRG']

In [ ]:
cormat = dataset[feature_cols].corr()
print('FRUITS DATASET - CORRELATION MATRIX: \n', round(cormat,2))
sns.heatmap(cormat);

In [ ]:
# Como primer análisis para mutual_info, seleccionamos aquellas features con un valor mayor 0.75
sns.pairplot(dataset[feature_cols]);

In [ ]:
x = dataset[feature_cols]
print('FRUITS DATASET - X: \n', x)
y = dataset.Class
print('FRUITS DATASET - Y: \n', y)

**Balanceo de Set de Datos**

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

over_sampler = RandomOverSampler(random_state=42)
X_res, y_res = over_sampler.fit_resample(x, y)

print(f"Training target statistics: {Counter(y_res)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({0: 204, 3: 204, 6: 204, 1: 204, 4: 204, 5: 204, 2: 204})
Testing target statistics: Counter({0: 60, 4: 45, 1: 40, 6: 38, 2: 36, 5: 35, 3: 32})


**Separación Set de Datos**

In [ ]:
set_test_size = 0.2

In [ ]:
# Obtenemoslos set de datos de entrenamiento y pruebas.
x_train, x_test, y_train, y_test = train_test_split(X_res, y_res, test_size=set_test_size, random_state=0, shuffle=True)

In [ ]:
x_train = x_train.values

In [ ]:
x_train

In [ ]:
y_train = y_train.values

In [ ]:
y_train

array([0, 3, 6, ..., 2, 4, 5])

In [ ]:
x_test = x_test.values

In [ ]:
x_test

In [ ]:
y_test = y_test.values

In [ ]:
y_test

**Normalizacion Modelo**

In [ ]:
data = pd.DataFrame(x_train, columns=feature_cols)

In [ ]:
z_scores = (data - data.mean()) / data.std()
z_scores

In [ ]:
nuevo_x_train = z_scores.values
nuevo_x_train

In [ ]:
data = pd.DataFrame(x_test, columns=feature_cols)

In [ ]:
z_scores = (data - data.mean()) / data.std()
z_scores

In [ ]:
nuevo_x_test = z_scores.values
nuevo_x_test

**Definición Modelo**

In [ ]:
regressor = LogisticRegression(max_iter=8000)

**Entrenamiento Modelo**

In [ ]:
regressor.fit(nuevo_x_train, y_train)

**Predicción Resultados**

In [ ]:
y_pred = regressor.predict(nuevo_x_test)

In [ ]:
print('FRUITS DATASET - Y PREDICTIONS: \n', y_pred)

**Evaluación Modelo**

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print('FRUITS DATASET - CONFUSION MATRIX: \n', cnf_matrix)

In [ ]:
# Visualizamos matriz de confusión.
class_names = [0, 1, 2, 3, 4, 5, 6]
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap='Blues_r', fmt='g')
ax.xaxis.set_label_position('top')
plt.tight_layout()
plt.title('Confusion Matrix', y=1.1)
plt.ylabel('Current Label')
plt.xlabel('Predicted Label')

In [ ]:
# Ejecutamos Reporte de Clasificación.
from sklearn.metrics import classification_report

print('FRUITS DATASET - CLASSIFICATION REPORT: \n', classification_report(y_test, y_pred))

In [ ]:
# Evaluamos Score.
print('FRUITS DATASET - ACCURACY SCORE BY METRICS: ', metrics.accuracy_score(y_test, y_pred))

In [ ]:
print('FRUITS DATASET - SCORE BY REGRESSOR SCORE: ', regressor.score(nuevo_x_test, y_test))

FRUITS DATASET - SCORE BY REGRESSOR SCORE:  0.8356643356643356
